In [11]:
#Try with bad data -- 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

############Configurable Parameters#############
FileName = "TOA5_ExProcessing"
InputPath = "FromPramod/Staton8/bad/"  #"FromPramod/Staton8/good/TOA5_50453_CO2_44_2023_235_0000.dat"
InputFileExt = ".dat"
LengthToPlot = 86400  #Max = 86400
WhichColumnToDisplay = "CO2"  #"CO2"  #BattVolt 


############COnstant Parameters#############
OutputPath = InputPath + "/OutFiles/"
OutputFileExt = ".csv"
OutputCleanedExt = "_Cleaned.csv"
MaxLength = 86400


#Read Input file and convert to csv
df = pd.read_csv(InputPath + FileName + InputFileExt)
# df = pd.read_csv("FromPramod/Station3/TOA5_50455_CO2_56_2023_257_0000.dat")

#check if output path exists , if no create it
if not (os.path.exists(OutputPath)):
    os.makedirs(OutputPath)
    print("Creating New Directory- OutFiles")

df.to_csv(OutputPath + FileName + OutputFileExt)

df2 = pd.read_csv(OutputPath + FileName + OutputFileExt,header=None)
# print(df2.head(2))

#Drop index 1,3,4 -- ie, 0,2,3
df2_drop = df2.drop([df2.index[0], df2.index[2],df2.index[3]])
# print(df2_drop.head(4))

#Keep 200 lines
RowsToKeep = 200



#Assign new header as columns
new_header = df2_drop.iloc[0]
df2_drop = df2_drop[1:]
df2_drop.columns = new_header
df_new = df2_drop.reset_index(drop=True)
# print(df2_drop.head(4))
# print(df_new.head(3))
#Get first 200 Rows
df_firstFew = df_new.head(RowsToKeep)

print(df_firstFew.size)
print(df_firstFew.tail(5))
df_firstFew.to_csv(OutputPath + FileName + "Stripped" + OutputFileExt)


/tmp/ipykernel_8650/3526851795.py:23: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(InputPath + FileName + InputFileExt)


5600
1              TIMESTAMP   RECORD  Year Month Day_of_Month Day_of_Week  \
195  2023-09-22 09:33:46  3569875  2023     9           22           6   
196  2023-09-22 09:33:47  3569876  2023     9           22           6   
197  2023-09-22 09:33:48  3569877  2023     9           22           6   
198  2023-09-22 09:33:49  3569878  2023     9           22           6   
199  2023-09-22 09:33:50  3569879  2023     9           22           6   

1   Day_of_Year Hour Minute Second  ...           CO2            H2O  \
195         265    9     33     46  ...  -0.007331962  -9.783333E-05   
196         265    9     33     47  ...  -0.008423512  -0.0004874438   
197         265    9     33     48  ...  -0.009412304   0.0001471807   
198         265    9     33     49  ...  -0.007638384  -0.0001321673   
199         265    9     33     50  ...  -0.008450584  -0.0002019428   

1   wind_speed wind_direction diag IncomingSW OutgoingSW IncomingLW  \
195       0.44             85    0      310.6 

/tmp/ipykernel_8650/3526851795.py:33: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(OutputPath + FileName + OutputFileExt,header=None)


In [19]:
#read from csv and 
df_Stripped = pd.read_csv(OutputPath + FileName + "Stripped" + OutputFileExt)

df_Stripped_drop6rows = df_Stripped.drop([2,3,4])
df_Stripped_dropped2 = df_Stripped.drop(df_Stripped.index[2:7])
# print(df_Stripped_drop6rows.head(10))
# print(df_Stripped_dropped2.head(10))
# print(df_Stripped.head(2))

GetDate1 = pd.to_datetime(df_Stripped["TIMESTAMP"][0])
print(GetDate1.date())
print(type(GetDate1.date()))
GetDate2 = GetDate1.date()
#create a new date time stamp 
dateTimeRange = pd.date_range(GetDate2,periods=86400, freq='s')
print(dateTimeRange[0:4])
#Get range of values

start_time = pd.Timestamp(str(GetDate2) + " 00:00:01")
end_time = pd.Timestamp(str(GetDate2) + " 00:00:00")
print(start_time)
print(end_time)
GetTimeRange = pd.date_range(start_time,periods=86400, freq='1s')
print(GetTimeRange[0:4])
print("Printing end date")
print(GetTimeRange[-5:])
#create a dataframe from GetTimeRange
df_ts = pd.DataFrame(index=GetTimeRange)

# print('***********')
df_TimeStamp = pd.to_datetime(df_Stripped_dropped2["TIMESTAMP"])
# df_Stripped_Reindxed = df_Stripped_dropped2.reindex(df_TimeStamp , method='ffill')
# df_Stripped_Reindxed2 = df_Stripped_dropped2.set_index(df_Stripped_dropped2["TIMESTAMP"])
df_Stripped_SetIndex = df_Stripped_dropped2.set_index(df_TimeStamp)
print(df_Stripped_SetIndex.head(3))

ColumnNamesFromStripped = df_Stripped.columns
# df_ts.columns = ColumnNamesFromStripped
combined_df = df_ts.combine_first(df_Stripped_SetIndex)
print("printing combined df")
# print(combined_df.head(3))

# Merge the two DataFrames, filling in NaN for missing values in df2
merged_df = pd.concat([df_Stripped_SetIndex, df_ts], axis=1, join='outer')
print("Printing merged df")
# print(merged_df.head(3))
# print(merged_df[merged_df.index["2023-09-22 09:29:58":"2023-09-22 09:30:58"]])
print(merged_df.loc["2023-09-22 09:29:58":"2023-09-22 09:30:58"])
merged_df.to_csv(OutputPath + FileName + "Merged" + OutputFileExt)
#Use set_index to set the index now
# df_Stripped_SetIndex2 = df_Stripped_dropped2.set_index(GetTimeRange)
# # df_Stripped_SetIndex2 = df_Stripped_dropped2.reset_index(GetTimeRange)
# print(df_Stripped_SetIndex2.head(3))

# # print(df_Stripped_Reindxed2.head())

# df_Stripped_Resample = df_Stripped_Reindxed2.resample('s').asfreq()
# print(df_Stripped_Resample.head())

# timestamp = pd.date_range()
#Check if the 

# print(type(df_Stripped["TIMESTAMP"][0]))
# df_Stripped_Clean = df_Stripped.loc[:,~df_Stripped.columns.str.contains('^Unnamed')]
# # print(df_Stripped_Clean.head(2))
# df_TimeStamp = df_Stripped["TIMESTAMP"]
# print(len(df_TimeStamp))


2023-09-22
<class 'datetime.date'>
DatetimeIndex(['2023-09-22 00:00:00', '2023-09-22 00:00:01',
               '2023-09-22 00:00:02', '2023-09-22 00:00:03'],
              dtype='datetime64[ns]', freq='S')
2023-09-22 00:00:01
2023-09-22 00:00:00
DatetimeIndex(['2023-09-22 00:00:01', '2023-09-22 00:00:02',
               '2023-09-22 00:00:03', '2023-09-22 00:00:04'],
              dtype='datetime64[ns]', freq='S')
Printing end date
DatetimeIndex(['2023-09-22 23:59:56', '2023-09-22 23:59:57',
               '2023-09-22 23:59:58', '2023-09-22 23:59:59',
               '2023-09-23 00:00:00'],
              dtype='datetime64[ns]', freq='S')
                     Unnamed: 0            TIMESTAMP   RECORD  Year  Month  \
TIMESTAMP                                                                    
2023-09-22 09:29:58           0  2023-09-22 09:29:58  3569647  2023      9   
2023-09-22 09:29:59           1  2023-09-22 09:29:59  3569648  2023      9   
2023-09-22 09:30:05           7  2023-09-22 

In [ ]:
# df_Stripped_Clean.index = pd.to_datetime(df_Stripped_Clean.index)
# GetDate = "2023-09-22"
# df_Stripped_Clean.TIMESTAMP = pd.to_datetime(df_Stripped_Clean.TIMESTAMP)
# df5min = df_Stripped_Clean.set_index('TIMESTAMP').resample('5Min')
# print(df5min)
# df_Stripped_Clean = df_Stripped_Clean.set_index('TIMESTAMP')
# df5min = df_Stripped_Clean.resample('5Min', on='TIMESTAMP').first().drop('TIMESTAMP', 1).reset_index()


# print(df_Stripped_Clean.head(2))
# df_Stripped_Clean_Timestamp = df_Stripped_Clean['TIMESTAMP'][2:]
# df_Stripped_Clean_ToTimestamp = pd.to_datetime(df_Stripped_Clean_Timestamp)
# df_Stripped_ReSample = df_Stripped_Clean_ToTimestamp.resample("1S").asfreq()
# df_Stripped_ReSample = df_Stripped_Clean_ToTimestamp.resample("1S").ffill()
# df_Stripped_ToTimestamp = pd.to_datetime(df_Stripped_Timestamp)
# print(type(df_Stripped_ToTimestamp[0]))
# df_Stripped_Resample = df_Stripped_ToTimestamp.resample("S").asfreq()
# print(df_Stripped_Resample.head(3))
# df['date'] = df_Stripped_ToTimestamp.date
# df_Stripped2 = df_Stripped[2:]
#Try resampling on df_Stripped
# df_Stripped_Resampled = df_Stripped2.resample("S").asfreq()
# df_Stripped_Timestamp.to_csv(OutputPath + FileName + "StrippedSampled" + OutputFileExt)
# print(df['date'][0])
# GetDate =  df_Stripped_ToTimestamp[0].date             #df_Stripped_Timestamp[0].dt.date
# print(GetDate)
# print(df_Stripped.head(2))
# print(df_Stripped.size)
# # print(df_Stripped_Timestamp.size)
# # print(df_Stripped_Timestamp.head(2))
# idx = pd.date_range('2018-01-01 00:00:01', periods=86400, freq='s')
# # print(idx)
# print(type(idx))
# print(idx[0])
# print(len(idx))


# LengthToPrint = 6
# count = 0
# for i in range(0,len(idx)):
#     if count < LengthToPrint:
#         print("i = {}, idx= {}".format(i, idx[i]))
#     count = count + 1




#Check the length of data = 86400 ie, 24*60*60 secs in a day
LengthOfData = df_new["TIMESTAMP"].count()
if( df_new["TIMESTAMP"].count() < MaxLength):
    TotalLengthOfBadData = df_new["TIMESTAMP"].count()
    print("Max length of data = {}".format(LengthToPlot))
    if LengthToPlot > TotalLengthOfBadData :
        LengthToPlot = TotalLengthOfBadData
    
elif df_new["TIMESTAMP"].count() == MaxLength:
    print("Length = 86400 -- Good data!")
else:
    print("Length > MaxLength -- Freq>1 Hz !!!")
    exit(-1)


# columnsToDrop = ["RECORD","Year", "Month","Day_of_Month","Day_of_Week","Day_of_Year","Hour","Minute","Second"]
columnsToDrop = ["RECORD","Year", "Month","Day_of_Month","Day_of_Week","Day_of_Year","Minute","Second"]
df_Clean = df_new.drop(columnsToDrop, axis=1)
# print(df_Clean.count())

df_Clean.to_csv(OutputPath + FileName + OutputCleanedExt)



#dataframes available : 
df_Clean = pd.read_csv(OutputPath + FileName + OutputCleanedExt)
# print(df_Clean.head(5))
#Delte first 

df_Clean = df_Clean.loc[:,~df_Clean.columns.str.contains('^Unnamed')]
print(df_Clean.head(5))

#Plot CO2
df_WhichColumn = df_Clean[WhichColumnToDisplay]
df_WhichColumn_Small = df_WhichColumn[:LengthToPlot]
df_WhichColumn_Small.plot()


plt.title(WhichColumnToDisplay + " Plot")
plt.savefig(OutputPath + FileName + "_" + WhichColumnToDisplay + "_Plot.png")
plt.show()

In [4]:
#Try with bad data -- 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os



############COnstant Parameters#############
OutputPath = InputPath + "/OutFiles/"
OutputFileExt = ".csv"
OutputCleanedExt = "_Cleaned.csv"
MaxLength = 86400

############Configurable Parameters#############
FileName = "TOA5_50453_CO2_77_2023_265_0929"
InputPath = "FromPramod/Staton8/bad/"  #"FromPramod/Staton8/good/TOA5_50453_CO2_44_2023_235_0000.dat"
InputFileExt = ".dat"
LengthToPlot = 86400  #Max = 86400
WhichColumnToDisplay = "CO2"  #"CO2"  #BattVolt 




#Read Input file and convert to csv
# df = pd.read_csv(InputPath + FileName + InputFileExt)
# # df = pd.read_csv("FromPramod/Station3/TOA5_50455_CO2_56_2023_257_0000.dat")

# #check if output path exists , if no create it
# if not (os.path.exists(OutputPath)):
#     os.makedirs(OutputPath)
#     print("Creating New Directory- OutFiles")

# df.to_csv(OutputPath + FileName + OutputFileExt)

df_Clean = pd.read_csv(OutputPath + FileName + OutputCleanedExt)
print(df_Clean.head(5))

# resampel
# df5min = df_Clean.resample('5Min', on='TIMESTAMP').first().drop('TIMESTAMP', 1).reset_index()

# print(df5min.head(5))
#Resample for 1 -sec data & vlaue = NaN 
df_Clean["TIMESTAMP"] = pd.to_datetime(df_Clean["TIMESTAMP"])



   Unnamed: 0            TIMESTAMP  BattVolt  PTemp  AirTemp     Rh  \
0           0  2023-09-22 09:29:58      8.17  23.28    27.21  23.63   
1           1  2023-09-22 09:29:59      8.17  23.28    27.07  23.29   
2           2  2023-09-22 09:30:00      8.17  23.28    27.17  23.41   
3           3  2023-09-22 09:30:01      8.18  23.28    27.08  23.16   
4           4  2023-09-22 09:30:02      8.18  23.28    27.07  23.16   

   SBTemp_1360  TargTemp_1360  TmV_1360    PAR       CO2       H2O wind_speed  \
0        29.65          24.47 -0.266155  767.2  0.004254 -0.000669       0.65   
1        29.65          24.38 -0.270545  769.3  0.000130 -0.000546       0.71   
2        29.65          24.34 -0.272572  768.2  0.002119 -0.000581        0.7   
3        29.65          24.34 -0.272376  765.1  0.000656 -0.000464       0.66   
4        29.65          24.33 -0.273166  762.9  0.001963  0.000136       0.75   

  wind_direction diag  IncomingSW  OutgoingSW  IncomingLW  OutgoingLW  Albedo  
0     

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'